In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('font',family='Malgun Gothic')
import seaborn as sns
import pandas as pd
import scipy.stats as stats

In [2]:
df_TOCC = pd.read_csv('../../datasets/TypeOfContractChannel.csv')
df_TOCC[:2]

,id,type_of_contract,type_of_contract2,channel,datetime,Term,payment_type,product,amount,state,overdue_count,overdue,credit rating,bank,cancellation,age,Mileage
0,66758234,렌탈,Normal,서비스 방문,2019-10-20,60,CMS,K1,96900,계약확정,0,없음,9.0,새마을금고,정상,43.0,1862.0
1,66755948,렌탈,Extension_Rental,서비스 방문,2019-10-20,60,카드이체,K1,102900,계약확정,0,없음,2.0,현대카드,정상,62.0,2532.0


#### X:연속형 / Y:연속형 : 주로 상관관계 확인
- correlation coefficient(상관계수)
    - -1 ~ 1
    - 0에 가까우면 관계없음, -1로 갈수록 반비례, 1로 갈수로 정비례
##### 선정 변수: age, Mileage
- 분석 내용 : 나이에 따라서 마일리지 변화가 있는가?
    - 귀무가설 : 나이에 따라서 마일리지 변화가 없다.
    - 대립가설 : 나이에 따라서 마일리지 변화가 있다.
- 분석결론 (기준:p-value 0.05) / P.value < 0.05 : 대립 참
    - 통계 결론 : 대립가설 참, 상관계수 비례관계
    - 사실 결론 : 나이에 따라서 마일리지 변화가 올라간다.(statistic=0.97, pvalue=0.0)

In [3]:
df_TOCC[['age','Mileage']].describe()

# count 
# mean : 연속형이라는 증거 

,age,Mileage
count,40509.000000,40509.000000
mean,46.608285,1967.922412
std,13.465977,436.707329
min,20.000000,872.000000
25%,37.000000,1646.000000
50%,46.000000,1946.000000
75%,56.000000,2263.000000
max,107.000000,3803.000000


In [7]:
df_age_Mileage = df_TOCC[['age','Mileage']]
df_age_Mileage[:2]

,age,Mileage
0,43.0,1862.0
1,62.0,2532.0


In [8]:
df_age_Mileage.isnull().sum()

age        10795
Mileage    10795
dtype: int64

In [9]:
df_age_Mileage = df_age_Mileage.dropna().copy()

In [10]:
df_age_Mileage.corr()

,age,Mileage
age,1.000000,0.973281
Mileage,0.973281,1.000000


In [11]:
stats.pearsonr(df_age_Mileage['age'],df_age_Mileage['Mileage'] )

# statistic : 상관계수값 
# pvalue : 현재 대립이 참 

PearsonRResult(statistic=0.9732809346109912, pvalue=0.0)

In [12]:
stats.pearsonr(df_age_Mileage['Mileage'],df_age_Mileage['age'] )

# 순서가 바뀌어도 값은 동일 

PearsonRResult(statistic=0.9732809346109912, pvalue=0.0)

In [13]:
# 컬럼 중 하나라도 비정규 분포이면 사용

stats.spearmanr(df_age_Mileage['Mileage'],df_age_Mileage['age'] )

SignificanceResult(statistic=0.972325776844677, pvalue=0.0)

In [14]:
df_TOCC[['type_of_contract', 'cancellation']].describe()

,type_of_contract,cancellation
count,51300,51279
unique,2,2
top,렌탈,정상
freq,46481,50372


In [15]:
df_TOCC_category = df_TOCC[['type_of_contract', 'cancellation']]

In [16]:
df_TOCC_category_pivot = pd.pivot_table(data=df_TOCC_category, index='type_of_contract' , columns='cancellation', aggfunc=len) # aggfunc=len -> count
df_TOCC_category_pivot

cancellation,정상,해약
type_of_contract,,
렌탈,46137,344
멤버십,4231,563


In [17]:
import scipy
scipy.__version__
import scipy.stats as stats

In [18]:
stats.chi2_contingency(observed=df_TOCC_category_pivot, correction=True) #data는 집계된 data로, correction는 자유도


Chi2ContingencyResult(statistic=3021.9735309482894, pvalue=0.0, dof=1, expected_freq=array([[45658.8007411,   822.1992589],
       [ 4709.1992589,    84.8007411]]))

In [19]:
# 다른 변수 이름 작성 시 copy() 사용
df_TOCC_product = df_TOCC[['product', 'cancellation']].copy()
df_TOCC_product[:2]

,product,cancellation
0,K1,정상
1,K1,정상


In [20]:
df_TOCC_product.describe() 
# count의 값이 차이가 있다는 것은 null값이 존재한다는 뜻
# unique 를 확인한 값을 통해 범주형임을 알수 있음 

,product,cancellation
count,51303,51279
unique,6,2
top,K1,정상
freq,39134,50372


In [21]:
df_TOCC_product.isnull().sum()

product          1
cancellation    25
dtype: int64

In [22]:
df_TOCC_product = df_TOCC[['cancellation', 'product']].dropna().copy()
df_TOCC_product.describe() 

,cancellation,product
count,51278,51278
unique,2,6
top,정상,K1
freq,50371,39111


In [23]:
df_TOCC_product_pivot = pd.pivot_table(data=df_TOCC_product, index='cancellation', columns='product', aggfunc=len)


In [24]:
## product
stats.chi2_contingency(observed=df_TOCC_product_pivot, correction=True) 

Chi2ContingencyResult(statistic=103.79474317337241, pvalue=8.372028115481391e-21, dof=5, expected_freq=array([[3.84192086e+04, 8.83393274e+03, 2.04517380e+03, 3.21216058e+02,
        6.33591306e+02, 1.17877452e+02],
       [6.91791353e+02, 1.59067261e+02, 3.68262023e+01, 5.78394243e+00,
        1.14086938e+01, 2.12254768e+00]]))

In [25]:
##  cancellation
stats.chi2_contingency(observed=df_TOCC_product_pivot, correction=True) 

Chi2ContingencyResult(statistic=103.79474317337241, pvalue=8.372028115481391e-21, dof=5, expected_freq=array([[3.84192086e+04, 8.83393274e+03, 2.04517380e+03, 3.21216058e+02,
        6.33591306e+02, 1.17877452e+02],
       [6.91791353e+02, 1.59067261e+02, 3.68262023e+01, 5.78394243e+00,
        1.14086938e+01, 2.12254768e+00]]))

In [26]:
df_TOCC_error = df_TOCC[['state', 'cancellation']]
#df_TOCC_error[:2]
df_TOCC_error.describe()

# count의 값이 차이가 있다는 것은 null값이 존재한다는 뜻
# unique 를 확인한 값을 통해 범주형임을 알수 있음 

,state,cancellation
count,51304,51279
unique,4,2
top,계약확정,정상
freq,50350,50372


In [27]:
df_TOCC_error.isnull().sum()


state            0
cancellation    25
dtype: int64

In [28]:
df_TOCC_error = df_TOCC[['state', 'cancellation']].dropna().copy()
df_TOCC_error.describe()

,state,cancellation
count,51279,51279
unique,4,2
top,계약확정,정상
freq,50326,50372


In [29]:
df_TOCC_error_pivot = pd.pivot_table(data=df_TOCC_error, index='state', columns='cancellation', aggfunc=len)
df_TOCC_error_pivot

cancellation,정상,해약
state,,
계약확정,50326.0,NaN
기간만료,32.0,NaN
해약진행중,14.0,NaN
해약확정,NaN,907.0


In [30]:
stats.chi2_contingency(observed=df_TOCC_category_pivot, correction=True)

Chi2ContingencyResult(statistic=3021.9735309482894, pvalue=0.0, dof=1, expected_freq=array([[45658.8007411,   822.1992589],
       [ 4709.1992589,    84.8007411]]))